In [1]:
import logging
import os

os.environ["PD_CLIENT_CREDENTIALS_PATH_ENV"] = "/Users/nisseknudsen/pd-2.pem"
os.environ["PD_CLIENT_STEP_API_KEY_ENV"] = "YYYY"
os.environ["PD_CLIENT_ORG_ENV"] = "XXXX"

logging.getLogger("pd.state.serialize").setLevel(logging.CRITICAL)

### Create an empty scenario with a sensor rig

In [2]:
from typing import List, Dict

from pd.state import PostProcessParams, Pose6D

from paralleldomain.generation.scenario import Scenario
from paralleldomain.mini_batch.sensor_rig import SensorRig, StepCameraSensor
from paralleldomain.model.annotation import AnnotationType
from paralleldomain.model.annotation import AnnotationTypes
from paralleldomain.model.type_aliases import SensorName


In [3]:
class CarSensorRig(SensorRig):
    def __init__(
        self,
        annotation_types: List[AnnotationType],
        camera_name: str = "Front",
        grayscale: bool = False,
    ):
        super().__init__()
        self._camera_name = camera_name
        self._annotation_types = annotation_types
        self._grayscale = grayscale

    @property
    def sensor_wise_available_annotations(self) -> Dict[SensorName, List[AnnotationType]]:
        return {self._camera_name: self._annotation_types}

    def get_sensors(self) -> List[StepCameraSensor]:
        sensor = SensorRig.create_camera_sensor(
            name=self._camera_name,
            width=768,
            height=768,
            frame_rate=10.0,
            field_of_view_degrees=70,
            pose=Pose6D.from_rpy_angles(
                x_metres=0.0, y_metres=0.0, z_metres=2.0, roll_degrees=0.0, pitch_degrees=0.0, yaw_degrees=0.0
            ),
            annotation_types=self._annotation_types,
            post_process_params=PostProcessParams(
                motion_blur_amount=0.0,
            ),
            grayscale=self._grayscale,
        )
        return [sensor]



In [4]:
scenario = Scenario(
    sensor_rig=CarSensorRig(annotation_types=[AnnotationTypes.InstanceSegmentation2D]),
)

### Create an environment with weather and time of day

In [5]:
from paralleldomain.generation.environment import Environment, Clouds, Rain, ConstantDistribution, Fog, Wetness, Night


In [6]:
environment = Environment(
    clouds=Clouds(distribution=ConstantDistribution(0.7)),
    rain=Rain(distribution=ConstantDistribution(1.0)),
    fog=Fog(),
    wetness=Wetness(distribution=ConstantDistribution(0.3)),
    time_of_day=Night(),
)

### Assign environment to scenario

In [7]:
scenario.set_environment(environment=environment)

Scenario(sensor_rig=<__main__.CarSensorRig object at 0x147ad9ea0>, random_seed=42, environment=Environment(clouds=<paralleldomain.generation.environment.Clouds object at 0x147ada770>, fog=<paralleldomain.generation.environment.Fog object at 0x147ad95a0>, rain=<paralleldomain.generation.environment.Rain object at 0x147adbee0>, wetness=<paralleldomain.generation.environment.Wetness object at 0x147ad8280>, time_of_day=<paralleldomain.generation.environment.Night object at 0x147adb310>), location=SF_6thAndMission_medium(name='SF_6thAndMission_medium', version='v1'), time_of_day='LS_sky_noon_mostlySunny_1250_HDS025', steps=[])

### Place ourselves (ego) in the scene

In [8]:
from paralleldomain.generation.simulation.pd.atomics.vehicle import EgoVehicleGenerator

scenario.add_agents(generator=EgoVehicleGenerator(asset_name="suv_medium_01"))


Scenario(sensor_rig=<__main__.CarSensorRig object at 0x147ad9ea0>, random_seed=42, environment=Environment(clouds=<paralleldomain.generation.environment.Clouds object at 0x147ada770>, fog=<paralleldomain.generation.environment.Fog object at 0x147ad95a0>, rain=<paralleldomain.generation.environment.Rain object at 0x147adbee0>, wetness=<paralleldomain.generation.environment.Wetness object at 0x147ad8280>, time_of_day=<paralleldomain.generation.environment.Night object at 0x147adb310>), location=SF_6thAndMission_medium(name='SF_6thAndMission_medium', version='v1'), time_of_day='LS_sky_noon_mostlySunny_1250_HDS025', steps=[EgoVehicleGenerator(asset_name='suv_medium_01', behavior=VehicleBehavior(velocity_start_min=8, velocity_start_max=12, velocity_target_min=12, velocity_target_max=15, gear_start=<VehicleGear.DRIVE: 'DRIVE'>), position=None)])

### Add other agents to scene

In [9]:
from paralleldomain.generation.simulation.pd.atomics.common import EgoRelativePosition
from paralleldomain.generation.simulation.pd.atomics.pedestrian import (
    RandomPedestrianGenerator,
    PedestrianGenerator,
    PedestrianBehavior,
    PedestrianMovement,
)


In [10]:
scenario.add_agents(generator=RandomPedestrianGenerator(count_min=0, count_max=0))
scenario.add_agents(
    generator=PedestrianGenerator(
        behavior=PedestrianBehavior(movement=PedestrianMovement.JAYWALKER),
        position=EgoRelativePosition(radius=10),
    )
)
# scenario.add_agents(generator=RandomPedestrianGenerator(count_min=10, count_max=20))

Scenario(sensor_rig=<__main__.CarSensorRig object at 0x147ad9ea0>, random_seed=42, environment=Environment(clouds=<paralleldomain.generation.environment.Clouds object at 0x147ada770>, fog=<paralleldomain.generation.environment.Fog object at 0x147ad95a0>, rain=<paralleldomain.generation.environment.Rain object at 0x147adbee0>, wetness=<paralleldomain.generation.environment.Wetness object at 0x147ad8280>, time_of_day=<paralleldomain.generation.environment.Night object at 0x147adb310>), location=SF_6thAndMission_medium(name='SF_6thAndMission_medium', version='v1'), time_of_day='LS_sky_noon_mostlySunny_1250_HDS025', steps=[EgoVehicleGenerator(asset_name='suv_medium_01', behavior=VehicleBehavior(velocity_start_min=8, velocity_start_max=12, velocity_target_min=12, velocity_target_max=15, gear_start=<VehicleGear.DRIVE: 'DRIVE'>), position=None), RandomPedestrianGenerator(velocity_min=0.5, velocity_max=4.0, count_min=0, count_max=0, radius=100.0, density=2.4, position=None), PedestrianGenerato

### Query data and write to disk

In [12]:
from paralleldomain.generation.stream import Stream
from paralleldomain.utilities.any_path import AnyPath
from paralleldomain.utilities.fsio import write_png

import matplotlib.pyplot as plt


for frame in Stream.frame_stream(
    scenario=scenario, number_of_frames=20, name="test", step_url="ssl://step-api.paralleldomain.com:3001"
):
    for camera_frame in frame.camera_frames:
        write_png(obj=camera_frame.image.rgba, path=AnyPath(f"{frame.frame_id}.png"))
        plt.imshow(camera_frame.image.rgb)


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x28b403f40> (for post_execute):


KeyboardInterrupt: 